In [1]:
!pip install -q pyomo
from pyomo.environ import *
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00


In [2]:
# total courses
n = 11

#credit array
c = np.array([3,2,6,5,5,4,6,8,8,5,4])

In [3]:
len(c)

11

In [4]:
model = ConcreteModel()
model.x = Var(np.arange(n), domain = Binary)
model.obj = Objective(expr= sum(model.x[i]*c[i] for i in range(n)), sense = maximize)
model.constraints = ConstraintList()
model.constraints.add(expr = sum(model.x[i]*c[i] for i in range(n)) >= 30)
model.constraints.add(expr = sum(model.x[i] for i in range(n)) >= 5)
model.constraints.add(expr = sum(model.x[i] for i in range(n)) <= 9)
model.constraints.add(expr = model.x[2] + model.x[7] <= 1)
model.constraints.add(expr = model.x[4] + model.x[9] <= 1)
model.constraints.add(expr = model.x[2] + model.x[8] <= 1)
model.constraints.add(expr = model.x[0] - model.x[10] >= 0)
model.constraints.add(expr = model.x[0] + model.x[1] + model.x[2] >= 1)
model.constraints.add(expr = model.x[0] + model.x[1] + model.x[2] <= 2)
model.constraints.add(expr = model.x[5] + model.x[6] <= 1)
model.constraints.add(expr = model.x[3] + model.x[6] + model.x[9] <= 2)

model.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None 

In [5]:
# install cbc solver
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

# solving your pipeModel using cbc solver
results = optCBC.solve(model)

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [6]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.042362213134765625
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [7]:
model.obj()

41.0

In [9]:
for i in range(n):
  if model.x[i].value > 0:
    print("C",i,": ",model.x[i].value)

C 0 :  1.0
C 1 :  1.0
C 3 :  1.0
C 4 :  1.0
C 6 :  1.0
C 7 :  1.0
C 8 :  1.0
C 10 :  1.0


In [10]:
#Relax the above-constructed formulation to a linear program and solve for optimality. Report the
#solution for the relaxed program and compare it with the solution MILP formulation.

In [11]:
model1 = ConcreteModel()
model1.x = Var(np.arange(n), domain = NonNegativeReals, bounds = (0,1))
model1.obj = Objective(expr= sum(model1.x[i]*c[i] for i in range(n)), sense = maximize)
model1.constraints = ConstraintList()
model1.constraints.add(expr = sum(model1.x[i]*c[i] for i in range(n)) >= 30)
model1.constraints.add(expr = sum(model1.x[i] for i in range(n)) >= 5)
model1.constraints.add(expr = sum(model1.x[i] for i in range(n)) <= 9)
model1.constraints.add(expr = model1.x[2] + model1.x[7] <= 1)
model1.constraints.add(expr = model1.x[4] + model1.x[9] <= 1)
model1.constraints.add(expr = model1.x[2] + model1.x[8] <= 1)
model1.constraints.add(expr = model1.x[0] - model1.x[10] >= 0)
model1.constraints.add(expr = model1.x[0] + model1.x[1] + model1.x[2] >= 1)
model1.constraints.add(expr = model1.x[0] + model1.x[1] + model1.x[2] <= 2)
model1.constraints.add(expr = model1.x[5] + model1.x[6] <= 1)
model1.constraints.add(expr = model1.x[3] + model1.x[6] + model1.x[9] <= 2)

model1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : NonNegativeReals
          1 :     0 :  None :     1 : False :  True : NonNegativeReals
          2 :     0 :  None :     1 : False :  True : NonNegativeReals
          3 :     0 :  None :     1 : False :  True : NonNegativeReals
          4 :     0 :  None :     1 : False :  True : NonNegativeReals
          5 :     0 :  None :     1 : False :  True : NonNegativeReals
          6 :     0 :  None :     1 : False :  True : NonNegativeReals
          7 :     0

In [12]:
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

# solving your pipeModel using cbc solver
results1 = optCBC.solve(model1)

In [13]:
print(results1)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.01390838623046875
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [14]:
model1.obj()

41.0

In [15]:
for i in range(n):
  print("C",i,": ",model1.x[i].value)

C 0 :  1.0
C 1 :  1.0
C 2 :  0.0
C 3 :  1.0
C 4 :  1.0
C 5 :  0.0
C 6 :  1.0
C 7 :  1.0
C 8 :  1.0
C 9 :  0.0
C 10 :  1.0


In [ ]:
#A faculty in IEOR lately proposed an advanced course as C11 with 9 credits. But this course clashes
#slot with C7, C8 and C9. Modify the above MILP formulation by adding this additional constraint.
#Solve your modified MILP for the optimal solution.
n = 12
newc = np.array([3,2,6,5,5,4,6,8,8,5,4,9])
model2 = ConcreteModel()
model2.x = Var(np.arange(n), domain = Binary)
model2.obj = Objective(expr= sum(model2.x[i]*newc[i] for i in range(n)), sense = maximize)
model2.constraints = ConstraintList()
model2.constraints.add(expr = sum(model2.x[i]*newc[i] for i in range(n)) >= 30)
model2.constraints.add(expr = sum(model2.x[i] for i in range(n)) >= 5)
model2.constraints.add(expr = sum(model2.x[i] for i in range(n)) <= 9)
model2.constraints.add(expr = model2.x[2] + model2.x[7] <= 1)
model2.constraints.add(expr = model2.x[4] + model2.x[9] <= 1)
model2.constraints.add(expr = model2.x[2] + model2.x[8] <= 1)
model2.constraints.add(expr = model2.x[0] - model2.x[10] >= 0)
model2.constraints.add(expr = model2.x[0] + model2.x[1] + model2.x[2] >= 1)
model2.constraints.add(expr = model2.x[0] + model2.x[1] + model2.x[2] <= 2)
model2.constraints.add(expr = model2.x[5] + model2.x[6] <= 1)
model2.constraints.add(expr = model2.x[3] + model2.x[6] + model2.x[9] <= 2)
#adding extra constraint
model2.constraints.add(expr = model2.x[7] + model2.x[11] <= 1)
model2.constraints.add(expr = model2.x[8] + model2.x[11] <= 1)
model2.constraints.add(expr = model2.x[9] + model2.x[11] <= 1)
model2.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 

In [ ]:
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

# solving your pipeModel using cbc solver
results2 = optCBC.solve(model2)

In [ ]:
print(results2)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 12
  Number of binary variables: 12
  Number of integer variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.019782304763793945
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
model2.obj()
for i in range(n):
  print("C",i,": ",model2.x[i].value)

C 0 :  1.0
C 1 :  1.0
C 2 :  0.0
C 3 :  1.0
C 4 :  1.0
C 5 :  0.0
C 6 :  1.0
C 7 :  1.0
C 8 :  1.0
C 9 :  0.0
C 10 :  1.0
C 11 :  0.0


In [ ]:
#Relax this modified MILP to a simple linear program and check if the modified MILP can be obtained
#by merely rounding the solution of the corresponding modified linear program.

In [ ]:
n=12
newc = np.array([3,2,6,5,5,4,6,8,8,5,4,9])

model3 = ConcreteModel()
model3.x = Var(np.arange(n), domain = NonNegativeReals, bounds = (0,1))
model3.obj = Objective(expr= sum(model3.x[i]*newc[i] for i in range(n)), sense = maximize)
model3.constraints = ConstraintList()
model3.constraints.add(expr = sum(model3.x[i]*newc[i] for i in range(n)) >= 30)
model3.constraints.add(expr = sum(model3.x[i] for i in range(n)) >= 5)
model3.constraints.add(expr = sum(model3.x[i] for i in range(n)) <= 9)
model3.constraints.add(expr = model3.x[2] + model3.x[7] <= 1)
model3.constraints.add(expr = model3.x[4] + model3.x[9] <= 1)
model3.constraints.add(expr = model3.x[2] + model3.x[8] <= 1)
model3.constraints.add(expr = model3.x[0] - model3.x[10] >= 0)
model3.constraints.add(expr = model3.x[0] + model3.x[1] + model3.x[2] >= 1)
model3.constraints.add(expr = model3.x[0] + model3.x[1] + model3.x[2] <= 2)
model3.constraints.add(expr = model3.x[5] + model3.x[6] <= 1)
model3.constraints.add(expr = model3.x[3] + model3.x[6] + model3.x[9] <= 2)
#adding extra constraint
model3.constraints.add(expr = model3.x[7] + model3.x[11] <= 1)
model3.constraints.add(expr = model3.x[8] + model3.x[11] <= 1)
model3.constraints.add(expr = model3.x[9] + model3.x[11] <= 1)
model3.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : NonNegativeReals
          1 :     0 :  None :     1 : False :  True : NonNegativeReals
          2 :     0 :  None :     1 : False :  True : NonNegativeReals
          3 :     0 :  None :     1 : False :  True : NonNegativeReals
          4 :     0 :  None :     1 : False :  True : NonNegativeReals
          5 :     0 :  None :     1 : False :  True : NonNegativeReals
          6 :     0 :  None :     1 : False :  True : NonNegativeReals
   

In [ ]:
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

results3 = optCBC.solve(model3)

In [ ]:
print(results3)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 10
  Error rc: 0
  Time: 0.025267601013183594
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
model3.obj()
for i in range(n):
  print("C",i,": ",model3.x[i].value)

C 0 :  1.0
C 1 :  1.0
C 2 :  0.0
C 3 :  1.0
C 4 :  1.0
C 5 :  0.0
C 6 :  1.0
C 7 :  1.0
C 8 :  1.0
C 9 :  0.0
C 10 :  1.0
C 11 :  0.0
